In [1]:
%load_ext autoreload
%autoreload 1
%aimport ecg_get_data
%aimport Models
%aimport train_test_validat
%aimport self_attention
%aimport ECGplot
import Models 
from train_test_validat import *
from self_attention import *
from  ecg_get_data import *
import matplotlib.pyplot as plt

import torch
import torch.utils.data as Data
import random

import time
random_seed = 1
torch.manual_seed(random_seed)    # reproducible
torch.cuda.manual_seed_all(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime()) 

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

data_path =  './npy_ECG/' #路径
lable_path = './label.npy'
model_path = './model/'+time_str
log_path = './log/'+  time_str


EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:1


In [3]:
x = load_data(data_path,EcgChannles_num=EcgChannles_num,EcgLength_num=EcgLength_num)
y = load_label(lable_path)

100%|██████████| 1698/1698 [00:09<00:00, 178.42it/s]


In [4]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(x,y,train_size=0.9,random_state = random_seed,shuffle = True,stratify=y)
print("         HTN     NHTN ")
print("train: %5d   %5d" % (train_y.sum(),len(train_y)-train_y.sum()))
print("test : %5d   %5d" % (test_y.sum(),len(test_y)-test_y.sum()))

         HTN     NHTN 
train:   628     900
test :    70     100


In [5]:
#train_dataset,valid_dataset = load_numpy_dataset_to_tensor_dataset(x,y,random_seed=random_seed,train_rate = 0.8)

In [6]:
BATCH_SIZE = 128
FOLDS = 1
EPOCHS = 2000  
PATIENCE = 64
LR = 0.00001

In [7]:
from torch.utils.tensorboard import SummaryWriter   
os.makedirs(model_path, exist_ok=True)
writer = SummaryWriter(log_path)
from torchsummary import summary

In [9]:
NET = [Models.CNN_ATT6() for i in range(FOLDS)]
print(summary(NET[0], (EcgChannles_num,EcgLength_num), device="cpu"))
writer.add_graph(NET[0],torch.rand([1,EcgChannles_num,EcgLength_num]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1             [-1, 12, 5000]              24
            Conv1d-2             [-1, 12, 5000]             156
              ReLU-3             [-1, 12, 5000]               0
       BatchNorm1d-4             [-1, 12, 5000]              24
           Dropout-5             [-1, 12, 5000]               0
            Conv1d-6             [-1, 12, 5000]             156
              ReLU-7             [-1, 12, 5000]               0
       BatchNorm1d-8             [-1, 12, 5000]              24
           Dropout-9             [-1, 12, 5000]               0
           Conv1d-10             [-1, 12, 5000]             156
             ReLU-11             [-1, 12, 5000]               0
      BatchNorm1d-12             [-1, 12, 5000]              24
          Dropout-13             [-1, 12, 5000]               0
          Softmax-14               [-1,

/workspace/data/Interpretable_HTN/self_attention.py:108: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  sita = np.array(sqrt(channels))
/workspace/data/Interpretable_HTN/self_attention.py:112: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  attn_matrix = (torch.bmm(q, k.permute(0,2 ,1)))/torch.from_numpy(sita)  #torch.bmm进行tensor矩阵乘法,q与k相乘得到的值为attn_matrix.
/workspace/data/Interpretable_HTN/self_attention.py:66: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't rec

In [10]:
from torch.optim.lr_scheduler import CosineAnnealingLR

for fold in range(FOLDS):
    
    early_stopping = EarlyStopping(PATIENCE, verbose=True, model_path=model_path, delta=0)
    train_dataset,valid_dataset = get_k_fold_dataset(fold=int(fold+1),x = train_x,y=train_y,k=FOLDS,random_seed = random_seed)
    train_dataloader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_dataloader = Data.DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

    NET[fold].to(DEVICE)
    optimizer  = torch.optim.Adam(NET[fold].parameters(), lr=LR)  
    criterion = torch.nn.CrossEntropyLoss()   
    #等间隔调整学习率
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max = 32)
    best_loss = 3
    for epoch in range(1,EPOCHS):
        time_all=0
        start_time = time.time()
        train_loss,train_acc = train_model(train_dataloader, NET[fold], criterion, optimizer,DEVICE) # 训练模型
        #scheduler.step() # 学习率迭代

        time_all = time.time()-start_time
        validate_loss,validate_acc = test_model(train_dataloader,criterion,NET[fold],DEVICE) # 测试模型
        writer.add_scalars(main_tag=str(fold)+'_Loss',tag_scalar_dict={'train': train_loss,'validate': validate_loss},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_Accuracy',tag_scalar_dict={'train': train_acc,'validate': validate_acc},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_LearningRate',tag_scalar_dict={'LR': optimizer.state_dict()['param_groups'][0]['lr']},global_step=epoch)

        print('- Epoch: %d - Train_loss: %.5f - Train_acc: %.5f - Val_loss: %.5f - Val_acc: %5f - T_Time: %.3f' %(epoch,train_loss,train_acc,validate_loss,validate_acc,time_all))
        print('当前学习率：%f' %optimizer.state_dict()['param_groups'][0]['lr'])

        if validate_loss < best_loss:
            best_loss = validate_loss
            print('Find better model in Epoch {0}, saving model.'.format(epoch))
            torch.save(NET[fold],  model_path+'/best_model_' + str(fold) + '.pt')  # 保存最优模型
        #是否满足早停法条件
        if(early_stopping(validate_loss,NET[fold])):
            print("Early stopping")
            break

    print('Fold %d Training Finished' %(fold+1))
    torch.cuda.empty_cache()# 清空显卡cuda
print('Training Finished')

1528
               HTN  NHTN
valid_dataset: 135  171
train_dataset: 493  1087
- Epoch: 1 - Train_loss: 0.67844 - Train_acc: 0.58435 - Val_loss: 0.67369 - Val_acc: 0.598013 - T_Time: 3.165
当前学习率：0.000010
Find better model in Epoch 1, saving model.
Validation loss decreased (inf --> 0.673686).  Saving model ...
                    --------------------------------------------------

- Epoch: 2 - Train_loss: 0.65665 - Train_acc: 0.60181 - Val_loss: 0.68189 - Val_acc: 0.611272 - T_Time: 1.179
当前学习率：0.000010
EarlyStopping counter: 1 out of 64

- Epoch: 3 - Train_loss: 0.63983 - Train_acc: 0.63136 - Val_loss: 0.66376 - Val_acc: 0.650603 - T_Time: 1.082
当前学习率：0.000010
Find better model in Epoch 3, saving model.
Validation loss decreased (0.673686 --> 0.663762).  Saving model ...
                    --------------------------------------------------

- Epoch: 4 - Train_loss: 0.62376 - Train_acc: 0.66337 - Val_loss: 0.62651 - Val_acc: 0.678996 - T_Time: 1.192
当前学习率：0.000010
Find better model in

# ------Display the attention value-----

# -----For Leads -----

In [ ]:
test_model_path = "/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/model/20220907_091538/best_model_0.pt"
test_model = torch.load(test_model_path).to(DEVICE)
test_x = x[0:1698]
test_y = x[0:1698]
test_x = MAX_MIN_normalization_by_feactures(test_x)
test_x = torch.FloatTensor(test_x)  #turn numpy to tensor
test_y = torch.LongTensor(test_y)
test_dataset = Data.TensorDataset(test_x, test_y)
test_dataloader = Data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)
test_model.eval()

test_model.eval()
attention_value_timestep = np.zeros(EcgChannles_num,)
for i,data in enumerate(test_dataloader,0):
    inputs,labels = data[0].to(DEVICE),data[1].to(DEVICE)
    outputs = test_model(inputs)
    #print(outputs)
    _,pred = outputs.max(1) # 求概率最大值对应的标签
    #print("the label :{labels},pred is {pred}".format(labels=labels[0],pred=pred[0]))
    attention_value = test_model.attention_value
    attention_value_timestep += (((attention_value.to('cpu'))[0]).detach().numpy()).sum(axis=0)
plt.figure(figsize=(5,5))
plt.stem(np.arange(0,len(attention_value_timestep)), attention_value_timestep)
plt.xticks(np.arange(0,len(attention_value_timestep)),['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
plt.ylabel("Sum of attention value")
plt.show()